Test du projet de class trajets indirect

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

In [3]:
%time df_3semaines=t.ouvrir_fichier_lapi('2019-01-29 00:00:00','2019-02-17 23:59:59').set_index('created').sort_index()

Wall time: 1min 30s


In [12]:
df_journee=t.ouvrir_fichier_lapi('2019-01-29','2019-01-31')

In [13]:
df_journee

,camera_id,created,immat,fiability,l,state
0,5,2019-01-29 00:00:04,9713b42132193b69c5a68909f0ee2630a06bc8e0,99,0,FR
1,5,2019-01-29 00:00:26,2e96ca72f9937b680579a3c088cc7bbb05c6dade,99,0,FR
2,5,2019-01-29 00:00:29,f94e283cc61de5979326ff065e511672c76bfb92,79,0,IT
3,5,2019-01-29 00:00:41,85bb2101e7da6aa3e8e227cf741dfdb2e1bacf25,99,0,FR
4,5,2019-01-29 00:01:26,a6736ef258c879e7fa724bdc54f66a7a10f7264c,58,0,FR
5,5,2019-01-29 00:01:57,c6100aed9a9ca88a839b68c367775e0785d034e6,88,0,UK
6,5,2019-01-29 00:02:28,d83c69b3f3e3a0918cf9b9a82e8af69afe93993e,48,0,FR
7,5,2019-01-29 00:02:32,68bd98395a9ff2febe7ef6cfc62e57865bd3b6b5,99,0,ES
8,5,2019-01-29 00:03:08,be95f432f37e04073ff6f14cd8dd853cd3270646,99,0,FR
9,5,2019-01-29 00:03:23,1a8e8df362d131d8741913271b0cf2240e2204ee,99,0,RO


In [745]:
%time df_3semaines.loc[pd.to_datetime('2019-01-29'):pd.to_datetime('2019-01-30')]

Wall time: 0 ns


,camera_id,immat,fiability,l,state
created,,,,,
2019-01-29 00:00:02,13,8b1b888305c26503d9b83b38ea6516402d985c5f,99,-1,BE
2019-01-29 00:00:03,8,c0dd4662304e0901a0866dabc8de414d8e19206d,99,0,IT
2019-01-29 00:00:03,14,d17d965f7b65bfb2fd814867bd3c714968d07328,99,-1,PT
2019-01-29 00:00:04,5,9713b42132193b69c5a68909f0ee2630a06bc8e0,99,0,FR
2019-01-29 00:00:05,10,1786fdedee0acfb8629e9b5cd61104d77bce066f,39,0,ES
2019-01-29 00:00:05,3,b9faa07d52299826c97a48b9974cdf7e17bf55d8,99,0,FR
2019-01-29 00:00:06,3,332a381c558c51acef76ab04873f438bd530015d,99,0,FR
2019-01-29 00:00:10,3,a099baf2aac15755c014ba2593797dcd903b95fb,89,0,FR
2019-01-29 00:00:13,3,13c9b885389ad4d59feaaac9eec0b330349c33f4,99,0,IT


LISTE DES TRAJETS

In [3]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':14, 'cam_d':11, 'trajets':[
                                                        {'cameras':[14,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[14,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':14, 'cam_d':5,'trajets':[{'cameras':[14,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[14,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':14, 'cam_d':9,'trajets':[{'cameras':[14,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[14,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':14, 'cam_d':7,'trajets':[{'cameras':[14,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[14,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A630','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A630','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A630','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A630','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A630','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A630','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A630','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [730]:
%time test=t.trajet(df_journee, '2019-01-29 08:00:00', 60, 16, [6,13], type='Global')

Wall time: 6.33 s


In [14]:
test=t.transit_1_jour(df_journee,'2019-01-29 06:00:00', liste_trajet_locale)

date : 2019-01-29 06:00:00 debut_traitement : 2019-04-04 11:37:56.280046
date : 2019-01-29 07:00:00 debut_traitement : 2019-04-04 11:38:30.202866
date : 2019-01-29 08:00:00 debut_traitement : 2019-04-04 11:39:24.455723
date : 2019-01-29 09:00:00 debut_traitement : 2019-04-04 11:40:30.992822
date : 2019-01-29 10:00:00 debut_traitement : 2019-04-04 11:41:39.863963
date : 2019-01-29 11:00:00 debut_traitement : 2019-04-04 11:42:46.032074
date : 2019-01-29 12:00:00 debut_traitement : 2019-04-04 11:43:47.519637
date : 2019-01-29 13:00:00 debut_traitement : 2019-04-04 11:44:49.935327


In [17]:
#MATRICE O-D !!II!
pd.pivot_table(test,values='l', index='origine', columns='destination',aggfunc='count', margins=True )

destination,A10,A62,A63,A630,A89,N10,All
origine,,,,,,,
A10,NaN,198.0,44.0,119.0,12.0,NaN,373
A62,57.0,NaN,2.0,57.0,7.0,127.0,250
A63,26.0,NaN,NaN,NaN,6.0,154.0,186
A630,NaN,65.0,NaN,NaN,87.0,828.0,980
A89,19.0,27.0,26.0,112.0,NaN,70.0,254
N10,NaN,195.0,504.0,329.0,47.0,NaN,1075
All,102.0,485.0,576.0,617.0,159.0,1179.0,3118


In [29]:
#df des immat 
test.loc[(test['cameras'].apply(lambda x : x[0]==6)) & (pd.DatetimeIndex(test['date_cam_1']).hour<10) & (pd.DatetimeIndex(test['date_cam_1']).hour>=9)]

,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours,o_d,origine,destination
13,2019-01-29 09:00:22,a6f25a7beadd8cc085bc02bb1a7239c0b61d5923,1,"(6, 3, 13)",2019-01-29 10:17:33,01:17:11,N10-A63,N10,A63
48,2019-01-29 09:01:18,7a1ea264891bd38655faa40e5a6d1e77a5aaa47c,1,"(6, 3, 18, 13)",2019-01-29 10:18:44,01:17:26,N10-A63,N10,A63
76,2019-01-29 09:02:05,c6d9a99953f46ee1e5bb203477e5cd28503d54b3,1,"(6, 3, 18, 13)",2019-01-29 10:20:29,01:18:24,N10-A63,N10,A63
91,2019-01-29 09:02:47,5425c8d74137c458d500bc297e603fb44290b82b,1,"(6, 3, 13)",2019-01-29 10:20:57,01:18:10,N10-A63,N10,A63
107,2019-01-29 09:03:13,bf9a54ef67c294135b5cb83e898b7794dd1a5f63,1,"(6, 3, 18, 13)",2019-01-29 10:21:38,01:18:25,N10-A63,N10,A63
139,2019-01-29 09:04:04,76139581dfdab8f6bda4b41b886e9a1453fd7e37,1,"(6, 3, 18, 13)",2019-01-29 11:35:51,02:31:47,N10-A63,N10,A63
144,2019-01-29 09:04:16,06bea1db519fc13a43d174ed3012bda3450af670,1,"(6, 3, 18, 13)",2019-01-29 10:22:57,01:18:41,N10-A63,N10,A63
168,2019-01-29 09:05:03,c1a51439c31610752dc5de32835c14ceac781838,1,"(6, 3, 18, 13)",2019-01-29 10:20:42,01:15:39,N10-A63,N10,A63
179,2019-01-29 09:05:18,d9358a39055f08db6abdfc0f5b77bb9236c2dcf3,1,"(6, 3, 18, 13)",2019-01-29 10:23:12,01:17:54,N10-A63,N10,A63
188,2019-01-29 09:05:33,2e3e30115f88059510d07257ba0b4dd8520b85a7,1,"(6, 3, 18, 13)",2019-01-29 10:25:38,01:20:05,N10-A63,N10,A63


In [34]:
date_fin=(pd.to_datetime('2019-01-28')+pd.Timedelta(days=2)).strftime('%Y-%m-%d')

In [35]:
date_fin

'2019-01-30'

,date_cam_1,immat,l,cameras,date_cam_2,tps_parcours
475,2019-01-29 08:15:57,565319bccae6af27bea3882d6950954f110c08ea,1,"(8, 5)",2019-01-29 08:46:29,00:30:32
548,2019-01-29 08:18:45,408b6da06c70d3e0e633aa9bc167b63f86118a95,1,"(8, 5)",2019-01-29 08:42:10,00:23:25
680,2019-01-29 08:23:12,f0315689d509f01da93b128ab687d4b506806a00,1,"(8, 5)",2019-01-29 08:51:41,00:28:29
721,2019-01-29 08:24:21,55073f56d0f5699e47cfe4108522eca7d7058ad8,1,"(8, 5)",2019-01-29 11:47:19,03:22:58
1024,2019-01-29 08:35:19,f27057ee9647b01ecd9d79d3c6ed520a5a942499,1,"(8, 5)",2019-01-29 13:05:30,04:30:11
1119,2019-01-29 08:38:40,7466c759dfa6d61602fa520744b487a00e441ad8,1,"(8, 5)",2019-01-29 10:49:02,02:10:22
1183,2019-01-29 08:41:14,c1e16a598c646e59e60dd80eaf4fb7de77ea4b4e,1,"(8, 5)",2019-01-29 12:30:22,03:49:08
1361,2019-01-29 08:48:20,ae7dbeba189d80635617310239ed620243322fd3,1,"(8, 5)",2019-01-29 11:14:07,02:25:47
1468,2019-01-29 08:52:12,93d2f35c3325cc48feaf8264c7316a8e8f0c746e,1,"(8, 5)",2019-01-29 10:04:23,01:12:11
